**Sanskrit OCR using CV2 and CNN.**

In [25]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout
import pickle

In [26]:
# Load the dataset
with open("dev_letter_D.p", "rb") as file:
    dataset = pickle.load(file, encoding='latin1')

In [27]:
print("Number of letter images in the dataset are: " + str(len(dataset)))  # Fixed variable name 'db' to 'dataset'

Number of letter images in the dataset are: 7702


In [28]:
# Separate the data into images and labels
images = [entry[0] for entry in dataset]
labels = [entry[1] for entry in dataset]

In [29]:
# Preprocess the images
# Resize and normalize the images, and convert to grayscale
images = [cv2.resize(image, (32, 32)) for image in images]
images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]
images = np.array(images, dtype=np.float32) / 255.0  # Normalize to the range [0, 1]

In [30]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [31]:
# Convert your data to NumPy arrays and one-hot encode the labels
X_train = np.array(X_train).reshape(-1, 32, 32, 1)  # Reshape for CNN
X_test = np.array(X_test).reshape(-1, 32, 32, 1)
y_train = to_categorical(y_train, num_classes=602)  # Assuming 602 classes
y_test = to_categorical(y_test, num_classes=602)  # Assuming 602 classes

In [32]:
# Define a simple convolutional neural network (CNN) model
model = keras.Sequential([
    layers.Input(shape=(32, 32, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(602, activation='softmax')  # 602 classes for Sanskrit characters
])

In [33]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Changed loss to 'categorical_crossentropy'

In [34]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
193/193 [==============================] - 2s 6ms/step - loss: 5.1739 - accuracy: 0.0928 - val_loss: 4.6785 - val_accuracy: 0.1622
Epoch 2/20
193/193 [==============================] - 1s 5ms/step - loss: 4.1428 - accuracy: 0.2355 - val_loss: 3.8431 - val_accuracy: 0.2784
Epoch 3/20
193/193 [==============================] - 1s 6ms/step - loss: 3.3125 - accuracy: 0.3590 - val_loss: 3.2270 - val_accuracy: 0.4244
Epoch 4/20
193/193 [==============================] - 1s 6ms/step - loss: 2.6679 - accuracy: 0.4759 - val_loss: 2.8535 - val_accuracy: 0.5029
Epoch 5/20
193/193 [==============================] - 1s 5ms/step - loss: 2.1822 - accuracy: 0.5577 - val_loss: 2.6201 - val_accuracy: 0.5756
Epoch 6/20
193/193 [==============================] - 1s 6ms/step - loss: 1.7945 - accuracy: 0.6265 - val_loss: 2.5219 - val_accuracy: 0.5938
Epoch 7/20
193/193 [==============================] - 1s 5ms/step - loss: 1.4983 - accuracy: 0.6859 - val_loss: 2.3881 - val_accuracy: 0.6535
Epoch 

In [37]:
#Save the model
model.save("sanskrit_letters.model")

In [38]:
#Download the model folder as .zip
!zip -r /content/sanskrit_letters.zip /content/sanskrit_letters.model

  adding: content/sanskrit_letters.model/ (stored 0%)
  adding: content/sanskrit_letters.model/saved_model.pb (deflated 87%)
  adding: content/sanskrit_letters.model/variables/ (stored 0%)
  adding: content/sanskrit_letters.model/variables/variables.index (deflated 59%)
  adding: content/sanskrit_letters.model/variables/variables.data-00000-of-00001 (deflated 31%)
  adding: content/sanskrit_letters.model/fingerprint.pb (stored 0%)
  adding: content/sanskrit_letters.model/keras_metadata.pb (deflated 87%)
  adding: content/sanskrit_letters.model/assets/ (stored 0%)


In [35]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

49/49 [==============================] - 0s 5ms/step - loss: 2.7110 - accuracy: 0.7372
Test accuracy: 0.7371836304664612


In [36]:
# Use the model for character recognition
# You can use the model to predict the class of a character in a new image
# For example:
sample_image = cv2.imread("12.png")
sample_image = cv2.resize(sample_image, (32, 32))
sample_image = cv2.cvtColor(sample_image, cv2.COLOR_BGR2GRAY)
sample_image = np.array(sample_image) / 255.0
prediction = model.predict(np.expand_dims(sample_image, axis=0))
predicted_class = np.argmax(prediction)

# Print the English class annotation for the predicted class
print(f"Predicted class: {predicted_class} - {dataset[predicted_class][2]}")

1/1 [==============================] - 0s 115ms/step
Predicted class: 90 - kaM
